# Mineração de padrões frequentes

> Esse projeto usa a versão 0.17.0 do mlxtend

In [22]:
import mlxtend
print(mlxtend.__version__)

0.17.0


### Bibliotecas básicas e outros imports

In [23]:
import warnings
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 200)
np.set_printoptions(suppress=True, precision=3)

### Carregamento dos dados

In [24]:
df = pd.read_csv('dataset/preprocess_acidentes_rodovias_brasileiras_2007_a_2021.csv', low_memory=False)
df['veiculos'] = df['veiculos'].apply(lambda x : str(x))
print(f'Linhas: {df.shape[0]} | Coluna:  {df.shape[1]}')
df.head(1)

Linhas: 1898820 | Coluna:  25


,id,dia_semana,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,pessoas,mortos,ilesos,ignorados,feridos,veiculos,data,mes,ano,severidade_acidente,estacao
0,1035523.0,domingo,SC,282,513.7,Xaxim,velocidade incompativel,saida de pista,com vitimas feridas,pleno dia,crescente,sol,simples,curva,2,0,1,0,1,1,2012-01-01,Jan,2012,feridos,verão


### Seleção de features

In [25]:
cols = ['classificacao_acidente',
        'causa_acidente', 
        'tipo_acidente', 
        'fase_dia',	
        'sentido_via', 
        'condicao_metereologica', 
        'tipo_pista', 
        'tracado_via', 
        'veiculos', 
        'estacao']

data = df[cols]

### Agrupamento pela classificação do acidente

#### 1º cluster: Sem vítimas

In [26]:
sem_vitimas = data.query('classificacao_acidente=="sem vitimas"')
sem_vitimas.drop(['classificacao_acidente'], axis=1, inplace=True)
print(f'Linhas: {sem_vitimas.shape[0]} | Coluna:  {sem_vitimas.shape[1]}')

Linhas: 984719 | Coluna:  9


#### 2º cluster: Vítimas feridas

In [27]:
vitimas_feridas = data.query('classificacao_acidente=="com vitimas feridas"')
vitimas_feridas.drop(['classificacao_acidente'], axis=1, inplace=True)
print(f'Linhas: {vitimas_feridas.shape[0]} | Coluna:  {vitimas_feridas.shape[1]}')

Linhas: 817393 | Coluna:  9


#### 3º cluster: Vítimas fatais

In [28]:
vitimas_fatais = data.query('classificacao_acidente=="com vitimas fatais"')
vitimas_fatais.drop(['classificacao_acidente'], axis=1, inplace=True)
print(f'Linhas: {vitimas_fatais.shape[0]} | Coluna:  {vitimas_fatais.shape[1]}')

Linhas: 85048 | Coluna:  9


#### Enconder da base

In [29]:
def transactionEncoderData(dados:np.ndarray) -> pd.DataFrame:
    transEncoder = TransactionEncoder()
    a_data = transEncoder.fit(dados).transform(dados)
    dados = pd.DataFrame(a_data, columns=transEncoder.columns_)
    return dados*1

#### Itens frequentes

> Fpgrowth foi escolhido por ser mais eficiente

In [30]:
def getFrequentItemsets(encoderData:pd.DataFrame, minsup:float=0.3, useColnames:bool=True) -> pd.DataFrame:
    return fpgrowth(encoderData, min_support=minsup, use_colnames=useColnames)

#### Regras

In [31]:
def getAssociationRules(itemsets:pd.DataFrame, metric:str='lift', minThreshold:int=1) -> pd.DataFrame:
    return association_rules(itemsets, metric=metric, min_threshold=minThreshold)

### Acidentes sem vítimas

In [32]:
sem_vitimas = transactionEncoderData(sem_vitimas.values)
frequent_itemsets = getFrequentItemsets(sem_vitimas)
rules_sem_vitimas = getAssociationRules(frequent_itemsets)
rules_sem_vitimas.sort_values(by='lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(pleno dia),(2),0.611618,0.558529,0.368865,0.603096,1.079794,0.027258,1.112288
0,(2),(pleno dia),0.558529,0.611618,0.368865,0.660422,1.079794,0.027258,1.143718
3,(reta),(2),0.711196,0.558529,0.427526,0.601137,1.076286,0.030303,1.106824
2,(2),(reta),0.558529,0.711196,0.427526,0.765450,1.076286,0.030303,1.231313
8,(ceu claro),(reta),0.466580,0.711196,0.354938,0.760723,1.069639,0.023108,1.206985
9,(reta),(ceu claro),0.711196,0.466580,0.354938,0.499072,1.069639,0.023108,1.064864
10,(dupla),(reta),0.433331,0.711196,0.321288,0.741437,1.042522,0.013105,1.116960
11,(reta),(dupla),0.711196,0.433331,0.321288,0.451757,1.042522,0.013105,1.033609
4,(pleno dia),(decrescente),0.611618,0.490767,0.301073,0.492256,1.003033,0.000910,1.002932
5,(decrescente),(pleno dia),0.490767,0.611618,0.301073,0.613473,1.003033,0.000910,1.004799


> <p style='text-align: justify;'><font size=2.85>As regras demonstram que em casos de acidentes sem vítimas, são comuns ocorrências que envolvem 2 veículos em pleno dia ou em pistas com traçado reto. Temos também acidentes em pleno dia ou em pistas com traçado reto, ambos no sentido decrescente e em dias de céu claro ou em pista dupla, ambos com traçado reto.</font></p>

### Acidentes com vítimas feridas

In [33]:
vitimas_feridas = transactionEncoderData(vitimas_feridas.values)
frequent_itemsets = getFrequentItemsets(vitimas_feridas)
rules_vitimas_feridas = getAssociationRules(frequent_itemsets)
rules_vitimas_feridas.sort_values(by='lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(2),(reta),0.485206,0.683091,0.353152,0.727839,1.065508,0.021712,1.164418
1,(reta),(2),0.683091,0.485206,0.353152,0.516991,1.065508,0.021712,1.065806
4,(ceu claro),(reta),0.529979,0.683091,0.380233,0.717450,1.050299,0.018209,1.121602
5,(reta),(ceu claro),0.683091,0.529979,0.380233,0.556636,1.050299,0.018209,1.060125
2,(decrescente),(reta),0.479751,0.683091,0.327881,0.683441,1.000512,0.000168,1.001105
3,(reta),(decrescente),0.683091,0.479751,0.327881,0.479997,1.000512,0.000168,1.000472


> <p style='text-align: justify;'><font size=2.85>Em casos de acidentes com vítimas feridas, temos o envolvimento de 2 veículos em pista com traçado reto, além de ocorrências em dias de céu claro ou em sentido decrescente, ambos em vias de traçado reto.</font></p>

### Acidentes com vítimas fatais

In [34]:
vitimas_fatais = transactionEncoderData(vitimas_fatais.values)
frequent_itemsets = getFrequentItemsets(vitimas_fatais)
rules_vitimas_fatais = getAssociationRules(frequent_itemsets)
rules_vitimas_fatais.sort_values(by='lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(2),(simples),0.490147,0.704296,0.385676,0.786859,1.117227,0.040468,1.387361
5,(simples),(2),0.704296,0.490147,0.385676,0.547605,1.117227,0.040468,1.127009
14,(plena noite),(reta),0.485161,0.709446,0.364230,0.750739,1.058204,0.020034,1.165661
15,(reta),(plena noite),0.709446,0.485161,0.364230,0.513400,1.058204,0.020034,1.058032
2,(ceu claro),(reta),0.566057,0.709446,0.417858,0.738191,1.040517,0.016271,1.109793
3,(reta),(ceu claro),0.709446,0.566057,0.417858,0.588992,1.040517,0.016271,1.055802
6,(2),(reta),0.490147,0.709446,0.353283,0.720770,1.015961,0.005550,1.040551
7,(reta),(2),0.709446,0.490147,0.353283,0.497970,1.015961,0.005550,1.015583
8,(crescente),(simples),0.531159,0.704296,0.377834,0.711338,1.009999,0.003740,1.024395
9,(simples),(crescente),0.704296,0.531159,0.377834,0.536470,1.009999,0.003740,1.011457


> <p style='text-align: justify;'><font size=2.85>Em acidentes com vítimas fatais, as regras nos mostra o envolvimento de 2 veículos em pistas do tipo simples ou em via de traçado reto. Em dias de céu claro, são comuns ocorrências em pistas simples ou em vias com traçado reto ou em sentido crescente. Temos ocorrências em plena noite em vias de traçado reto, além de acidentes em pistas simples no sentido crescente ou em pistas com traçado reto também no sentido crescente.</font></p>